In [1]:
from google.cloud import bigquery
import json
import os
import pandas as pd

GCP_PROJECT_NAME = 'opensource-observer'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../oso_gcp_credentials.json'
bq_client = bigquery.Client(GCP_PROJECT_NAME)

In [2]:
onchain_builder_contracts = bq_client.query("""

    WITH submitted_apps AS (
      SELECT project_id,
      FROM `op_atlas.application`
      WHERE
        status = 'submitted'
        AND round_id = '8'
    ),
    
    chain_mappings AS (
      SELECT DISTINCT
        chain_id,
        CASE
          WHEN chain = 'op' THEN 'optimism'
          WHEN chain = 'fraxtal' THEN 'frax'
          ELSE chain
        END AS chain
      FROM `optimism_superchain_raw_onchain_data.transactions`
      WHERE dt BETWEEN '2025-02-24' AND '2025-03-03'
    ),
    
    latest_project_info AS (
      SELECT
        app.project_id,
        MAX_BY(proj.name, proj.updated_at) AS name,
        MAX_BY(proj.description, proj.updated_at) AS description
      FROM `op_atlas.project` AS proj
      JOIN submitted_apps AS app
        ON app.project_id = proj.id
      GROUP BY app.project_id
    ),
    
    all_primary_contracts AS (
      SELECT DISTINCT *
      FROM (
    
        -- First part: get contracts from project_contract (blockchain artifacts)
        SELECT
          lp.project_id,
          lp.name,
          pc.chain_id,
          UPPER(cm.chain) AS artifact_source,
          LOWER(pc.contract_address) AS artifact_name,
          1 AS discovery_method
        FROM latest_project_info lp
        JOIN `op_atlas.project_contract` pc
          ON lp.project_id = pc.project_id
        JOIN chain_mappings cm
          ON CAST(pc.chain_id AS STRING) = CAST(cm.chain_id AS STRING)
    
        UNION ALL
    
        SELECT
          cd.project_id,
          lp.name,
          chain_mappings.chain_id,
          UPPER(abp.artifact_source) AS artifact_source,
          LOWER(abp.artifact_name) AS artifact_name,
          2 AS discovery_method
        FROM latest_project_info lp
        JOIN (
          SELECT project_id, oso_project_name
          FROM UNNEST([
            STRUCT('0x0008577196fa6ec286440b418e2f6305fc10e62ce759625d826be272ee6b45a3' AS project_id, 'createx' AS oso_project_name),
            STRUCT('0x08df6e20a3cfabbaf8f34d4f4d048fe7da40447c24be0f3ad513db6f13c755dd', 'velodrome'),
            STRUCT('0x149bc5c396fd3dccac39c92164bb6b8bccc4d215321b58e241af07a76616e031', 'hop-protocol'),
            STRUCT('0x4288aacdda65af69c03b2c9b9d057e2b93ed1b7671e481b03c6a7b33349d2365', 'aerodrome-finance'),
            STRUCT('0x513265758a3804184b6a0771d99c1a4b04d3267bacd4b914e89ff991de229f6a', 'create2deployer'),
            STRUCT('0x517eaa9c56951de89261f2d7830ea49aae92f2a903104a17d9c5c2edd4959806', 'lifinance'),
            STRUCT('0x59fcf753703c0f32c9d601b7f0071304819e832262efc0c0903f932bb586d10b', 'givestation'),
            STRUCT('0x5b054742b09f8ce4092a15700ef9781520a6ff2928ba5ee09c78ad4dd8817b85', 'kiwi-news-attestate'),
            STRUCT('0x62e37e96aa6e1cbfb6bd24b97c4b8f1e12cc3fe35d5388d2f041c42a12b40745', 'stargate-finance'),
            STRUCT('0x808b31862cec6dccf3894bb0e76ce4ca298e4c2820e2ccbf4816da740463b220', 'fractal-visions'),
            STRUCT('0x8d1971ff2cf36055f986399fc91bff2d3bf815d2cff4f169f154fc70035fc193', 'ethdailyhub'),
            STRUCT('0x913ca97cfc8b587734059e4df9cc6de28ef590727e13e1b30ae9a57bea881731', 'apriloracle'),
            STRUCT('0xa88844cea135382e3484e39c3172033437121b35ca0bc8b10b9b8253984876b5', 'ethereum-attestation-service'),
            STRUCT('0xafca3591b83586f14e21b397135b95af668f5ebb54cb45053af44a73df7d7640', 'layerswap'),
            STRUCT('0xb6a587b5b85b1c70847442e3b4a84cecfb55aedd286628a85fbb12770ceb7018', 'travenesia'),
            STRUCT('0xe8833ceee8beb2b3fb0f7f2dcef576f6f9cf20e35d8e53324d71b0ed47b01982', 'socket'),
            STRUCT('0xed3d54e5394dc3ed01f15a67fa6a70e203df31c928dad79f70e25cb84f6e2cf9', 'host-it'),
            STRUCT('0xefaeef1446de875ab6e50d31c8dbe776f99d884cb8cb68d50b1384067e4aeb6b', 'synapse'),
            STRUCT('0xff1edf67227651293a9cff4dce3577c6ca52bf8c7c6f5f133f07b05255379b27', 'kelp-dao')
          ])
        ) AS cd
          ON lp.project_id = cd.project_id
        JOIN `metrics.artifacts_by_project_v1` abp
          ON abp.project_name = cd.oso_project_name
        JOIN chain_mappings
          ON lower(abp.artifact_source) = lower(chain_mappings.chain)
      )
    ),
    
    discovered_contracts AS (
      SELECT
        all_primary_contracts.project_id,
        all_primary_contracts.name,
        chain_mappings.chain_id,
        UPPER(chain_mappings.chain) AS artifact_source,
        c.contract_address AS artifact_name,
        3 AS discovery_method
      FROM `metrics.contracts_v0` c
      JOIN all_primary_contracts ON c.factory_address = all_primary_contracts.artifact_name
      JOIN chain_mappings ON all_primary_contracts.chain_id = chain_mappings.chain_id
    ),
    
    unioned_contracts AS (
      SELECT * FROM discovered_contracts
      UNION ALL
      SELECT * FROM all_primary_contracts
    ),
    
    all_contracts AS (
      SELECT
        project_id,
        name,
        chain_id,
        artifact_source AS chain,
        artifact_name AS contract_address,
        MIN(discovery_method) AS discovery_method
      FROM unioned_contracts
      GROUP BY 1,2,3,4,5
    )
    
    SELECT *
    FROM all_contracts

""")

df_onchain_builder_contracts = onchain_builder_contracts.to_dataframe()

In [3]:
df_onchain_builder_contracts.to_parquet('data/onchain_builder_contracts.parquet')

In [25]:
onchain_builder_activity = bq_client.query("""

    WITH submitted_apps AS (
      SELECT project_id,
      FROM `op_atlas.application`
      WHERE
        status = 'submitted'
        AND round_id = '8'
    ),
    
    chain_mappings AS (
      SELECT DISTINCT
        chain_id,
        CASE
          WHEN chain = 'op' THEN 'optimism'
          WHEN chain = 'fraxtal' THEN 'frax'
          ELSE chain
        END AS chain
      FROM `optimism_superchain_raw_onchain_data.transactions`
      WHERE dt BETWEEN '2025-02-24' AND '2025-03-03'
    ),
    
    latest_project_info AS (
      SELECT
        app.project_id,
        MAX_BY(proj.name, proj.updated_at) AS name,
        MAX_BY(proj.description, proj.updated_at) AS description
      FROM `op_atlas.project` AS proj
      JOIN submitted_apps AS app
        ON app.project_id = proj.id
      GROUP BY app.project_id
    ),
    
    all_primary_contracts AS (
      SELECT DISTINCT *
      FROM (
    
        -- First part: get contracts from project_contract (blockchain artifacts)
        SELECT
          lp.project_id,
          lp.name,
          pc.chain_id,
          UPPER(cm.chain) AS artifact_source,
          LOWER(pc.contract_address) AS artifact_name,
          1 AS discovery_method
        FROM latest_project_info lp
        JOIN `op_atlas.project_contract` pc
          ON lp.project_id = pc.project_id
        JOIN chain_mappings cm
          ON CAST(pc.chain_id AS STRING) = CAST(cm.chain_id AS STRING)
    
        UNION ALL
    
        SELECT
          cd.project_id,
          lp.name,
          chain_mappings.chain_id,
          UPPER(abp.artifact_source) AS artifact_source,
          LOWER(abp.artifact_name) AS artifact_name,
          2 AS discovery_method
        FROM latest_project_info lp
        JOIN (
          SELECT project_id, oso_project_name
          FROM UNNEST([
            STRUCT('0x0008577196fa6ec286440b418e2f6305fc10e62ce759625d826be272ee6b45a3' AS project_id, 'createx' AS oso_project_name),
            STRUCT('0x08df6e20a3cfabbaf8f34d4f4d048fe7da40447c24be0f3ad513db6f13c755dd', 'velodrome'),
            STRUCT('0x149bc5c396fd3dccac39c92164bb6b8bccc4d215321b58e241af07a76616e031', 'hop-protocol'),
            STRUCT('0x4288aacdda65af69c03b2c9b9d057e2b93ed1b7671e481b03c6a7b33349d2365', 'aerodrome-finance'),
            STRUCT('0x513265758a3804184b6a0771d99c1a4b04d3267bacd4b914e89ff991de229f6a', 'create2deployer'),
            STRUCT('0x517eaa9c56951de89261f2d7830ea49aae92f2a903104a17d9c5c2edd4959806', 'lifinance'),
            STRUCT('0x59fcf753703c0f32c9d601b7f0071304819e832262efc0c0903f932bb586d10b', 'givestation'),
            STRUCT('0x5b054742b09f8ce4092a15700ef9781520a6ff2928ba5ee09c78ad4dd8817b85', 'kiwi-news-attestate'),
            STRUCT('0x62e37e96aa6e1cbfb6bd24b97c4b8f1e12cc3fe35d5388d2f041c42a12b40745', 'stargate-finance'),
            STRUCT('0x808b31862cec6dccf3894bb0e76ce4ca298e4c2820e2ccbf4816da740463b220', 'fractal-visions'),
            STRUCT('0x8d1971ff2cf36055f986399fc91bff2d3bf815d2cff4f169f154fc70035fc193', 'ethdailyhub'),
            STRUCT('0x913ca97cfc8b587734059e4df9cc6de28ef590727e13e1b30ae9a57bea881731', 'apriloracle'),
            STRUCT('0xa88844cea135382e3484e39c3172033437121b35ca0bc8b10b9b8253984876b5', 'ethereum-attestation-service'),
            STRUCT('0xafca3591b83586f14e21b397135b95af668f5ebb54cb45053af44a73df7d7640', 'layerswap'),
            STRUCT('0xb6a587b5b85b1c70847442e3b4a84cecfb55aedd286628a85fbb12770ceb7018', 'travenesia'),
            STRUCT('0xe8833ceee8beb2b3fb0f7f2dcef576f6f9cf20e35d8e53324d71b0ed47b01982', 'socket'),
            STRUCT('0xed3d54e5394dc3ed01f15a67fa6a70e203df31c928dad79f70e25cb84f6e2cf9', 'host-it'),
            STRUCT('0xefaeef1446de875ab6e50d31c8dbe776f99d884cb8cb68d50b1384067e4aeb6b', 'synapse'),
            STRUCT('0xff1edf67227651293a9cff4dce3577c6ca52bf8c7c6f5f133f07b05255379b27', 'kelp-dao')
          ])
        ) AS cd
          ON lp.project_id = cd.project_id
        JOIN `metrics.artifacts_by_project_v1` abp
          ON abp.project_name = cd.oso_project_name
        JOIN chain_mappings
          ON lower(abp.artifact_source) = lower(chain_mappings.chain)
      )
    ),
    
    discovered_contracts AS (
      SELECT
        all_primary_contracts.project_id,
        all_primary_contracts.name,
        chain_mappings.chain_id,
        UPPER(chain_mappings.chain) AS artifact_source,
        c.contract_address AS artifact_name,
        3 AS discovery_method
      FROM `metrics.contracts_v0` c
      JOIN all_primary_contracts ON c.factory_address = all_primary_contracts.artifact_name
      JOIN chain_mappings ON all_primary_contracts.chain_id = chain_mappings.chain_id
    ),
    
    unioned_contracts AS (
      SELECT * FROM discovered_contracts
      UNION ALL
      SELECT * FROM all_primary_contracts
    ),
    
    all_contracts AS (
      SELECT
        project_id,
        name,
        chain_id,
        artifact_source AS chain,
        artifact_name AS contract_address,
        MIN(discovery_method) AS discovery_method
      FROM unioned_contracts
      GROUP BY 1,2,3,4,5
    ),

    filtered_activity AS (
        SELECT
            all_contracts.project_id,
            all_contracts.name,
            all_contracts.chain,
            txns.transaction_hash,
            txns.from_address_tx,
            txns.gas_fee_tx,
            txns.block_timestamp
          FROM `oso.int_superchain_traces_txs_joined` txns
          JOIN all_contracts
            ON txns.to_address_trace = all_contracts.contract_address
            AND txns.chain = all_contracts.chain
          WHERE
            txns.block_timestamp BETWEEN DATE_SUB('2025-02-28', INTERVAL 180 DAY) AND '2025-02-28'
    ),
    
    activity as (
      SELECT
        project_id,
        name,
        APPROX_COUNT_DISTINCT(transaction_hash) as num_txns,
        APPROX_COUNT_DISTINCT(from_address_tx) as distinct_users,
        APPROX_COUNT_DISTINCT(block_timestamp) as active_days,
        SUM(gas_fee_tx) as gas_fees,
        ARRAY_AGG(DISTINCT chain) as chains
      FROM filtered_activity
      GROUP BY 1,2
    )
    
    SELECT * 
    FROM activity

""")

df_onchain_builder_activity = onchain_builder_activity.to_dataframe()

In [29]:
df_onchain_builder_activity.to_csv('data/20250305_onchain_builders_eligibility.csv')